# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv('../output_data/cities.csv', index_col = False)
city_weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Make 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Lat", "Lng"]].astype(float)
humidity = city_weather_df["Humidity"].astype(float)

# Create Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down Melissa's best weather conditions for vacationing in
# A max temperature lower than 75 degrees but higher than 55.
# Wind speed less than 10 and moody clouds ;)
best_temp = city_weather_df ['Max Temp'] < 75
best_temp = city_weather_df ['Max Temp'] > 55
best_wind = city_weather_df ['Wind Speed'] < 10
best_cloudiness = city_weather_df ['Cloudiness'] > 60
best_fog = city_weather_df ['Humidity'] > 60
my_weather = best_temp & best_wind & best_cloudiness & best_fog

# Drop the rows with nulls
weirdo_vacations = city_weather_df[my_weather]
weirdo_vacations = weirdo_vacations.dropna()
weirdo_vacations = weirdo_vacations.reset_index()
weirdo_vacations.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
1,23,23,maragogi,97,BR,1558378610,69,-9.01,-35.22,83.94,9.42
2,45,45,rosita,72,NI,1558378762,71,13.92,-84.40,84.12,4.41
3,48,48,port alfred,67,ZA,1558378763,69,-33.59,26.89,61.00,8.01
4,91,91,bella vista,62,AR,1558378771,80,-28.51,-59.05,75.66,7.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel df and add a column for hotel name
hotel_df = pd.DataFrame(weirdo_vacations, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
hotel_df.head()

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,avarua,CK,-21.21,-159.78,71.60,NaN
1,maragogi,BR,-9.01,-35.22,83.94,NaN
2,rosita,NI,13.92,-84.40,84.12,NaN
3,port alfred,ZA,-33.59,26.89,61.00,NaN
4,bella vista,AR,-28.51,-59.05,75.66,NaN


In [7]:
counter = 0

for index, row in hotel_df.iterrows():

    # Set parameters to search for hotels with 5000 meters.
    params = {"location": f"{row['Lat']}, {row['Lng']}", "radius": 5000, "types": "lodging",
              "keyword": "Hotel", "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a json response with the parameters above
    response = requests.get(base_url, params=params)
    
    # convert to json and print them easy to read
    response_j = response.json()
    print(json.dumps(response_j, indent=4, sort_keys=True))
    
    results = response_j['results']
    
    # Name the hotel if you get a positive result
    if len(results) > 0:
        name = response_j['results'][0]['name']
        hotel_df.iloc[counter, -1] = name
    counter = counter + 1

hotel_weirdo_df = hotel_df
hotel_weirdo_df

NameError: name 'json' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_weirdo_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Plot marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(marker_layer)